In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('../data/after_transcription.csv', low_memory=False)

In [4]:
df.head()

,Unnamed: 0,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,is_inpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,Note
0,0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,24208 days,"PROCEDURE:, Upper endoscopy.,PREOPERATIVE DIA..."
1,1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,6186,1068.0,2009-09-02,750,6186,2948,56400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN,1.0,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,24349 days,"PREOPERATIVE DIAGNOSES: , Bilateral cleft lip ..."
2,2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,29590,1068.0,2009-09-20,883,29623,30390,71690,34590,V1581,32723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,24366 days,"ALLOWED CONDITIONS:, Lateral epicondylitis, r..."
3,3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,431,1068.0,2009-02-22,067,43491,2762,7843,32723,V1041,4254,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN,1.0,1914-03-01,NaN,2,2,0,1,360,12,12,2,1,1,2,2,1,1,2,2,1,1,5000,1068,250,320,34684 days,"HISTORY: , The patient is a 5-1/2-year-old, wh..."
4,4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,78321,1068.0,2009-08-30,975,042,3051,34400,5856,42732,486,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN,1.0,1938-04-01,NaN,2,1,Y,45,780,12,12,2,1,1,2,1,1,2,1,2,2,2,21260,2136,120,100,26067 days,"HISTORY OF PRESENT ILLNESS: , The patient is a..."


In [5]:
time_cols = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt', 'DOB']
for col in time_cols:
    df[col] = pd.to_datetime(df[col])

In [6]:
df.dtypes

Unnamed: 0                                  int64
BeneID                                     object
ClaimID                                    object
ClaimStartDt                       datetime64[ns]
ClaimEndDt                         datetime64[ns]
Provider                                   object
InscClaimAmtReimbursed                      int64
AttendingPhysician                         object
OperatingPhysician                         object
OtherPhysician                             object
AdmissionDt                        datetime64[ns]
ClmAdmitDiagnosisCode                      object
DeductibleAmtPaid                         float64
DischargeDt                        datetime64[ns]
DiagnosisGroupCode                         object
ClmDiagnosisCode_1                         object
ClmDiagnosisCode_2                         object
ClmDiagnosisCode_3                         object
ClmDiagnosisCode_4                         object
ClmDiagnosisCode_5                         object


In [7]:
df['Age'] = df['Age'].str.split(' ', expand=True)[0].astype(int)/365

In [8]:
df['Age'] = df['Age'].round()

In [9]:
df.head(1)

,Unnamed: 0,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,is_inpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,Note
0,0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,66.0,"PROCEDURE:, Upper endoscopy.,PREOPERATIVE DIA..."


In [10]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
df['Note'] = df['Note'].fillna('').astype(str)
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

df['Note'] = df['Note'].apply(clean_text)

In [12]:
df['note_length'] = df['Note'].str.len()

In [13]:
df['word_count'] = df['Note'].str.split().str.len()

In [14]:
df['sentence_count'] = df['Note'].str.count(r'[.!?]')

In [15]:
df.sample(2)['word_count']

508641    208
303694    144
Name: word_count, dtype: int64

In [16]:
medical_terms = [
    'surgery', 'procedure', 'incision', 'biopsy',
    'tumor', 'fracture', 'infection', 'chronic',
    'acute', 'diabetes', 'cardiac', 'renal',
    'malignant', 'therapy', 'anesthesia'
]

In [20]:
df["medical_term_count"] = df["Note"].apply(
    lambda x: sum(term in x for term in medical_terms)
)

In [21]:
df.sample()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,is_inpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,Note,note_length,word_count,sentence_count,medical_term_count
543025,BENE154743,CLM394126,2009-06-02,2009-06-02,PRV53790,200,PHY414991,PHY414991,PHY431212,NaT,NaN,0.0,NaT,NaN,25070,2808,5859,5853,2753,3572,78722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1930-12-01,NaN,1,1,Y,21,150,12,12,2,2,1,2,2,2,1,1,2,2,2,0,0,1270,800,79.0,procedure medial branch rhizotomy lumbosacral ...,3779,588,0,2


In [22]:
general_words = [
    "routine", "follow up",
    "as discussed", "as planned",
    "normal", "stable"
]

df["general_word_count"] = df["Note"].apply(
    lambda x: sum(phrase in x for phrase in general_words)
)

In [23]:
df["short_note_flag"] = (df["word_count"] < 10).astype(int)

In [24]:
high_cost_threshold = df["InscClaimAmtReimbursed"].quantile(0.90)
df["High_Cost_Flag"] = (
    df["InscClaimAmtReimbursed"] > high_cost_threshold
).astype(int)

In [25]:
df.sample()['High_Cost_Flag']

193497    0
Name: High_Cost_Flag, dtype: int64

In [26]:
df["highcost_shortnote"] = (
    df["High_Cost_Flag"] &
    df["short_note_flag"]
).astype(int)

In [27]:
diag_cols = [col for col in df.columns if "ClmDiagnosisCode_" in col]
df["diag_count"] = df[diag_cols].notna().sum(axis=1)

In [28]:
proc_cols = [col for col in df.columns if "ClmProcedureCode_" in col]
df["procedure_count"] = df[proc_cols].notna().sum(axis=1)

In [29]:
df["ClaimStartDt"] = pd.to_datetime(df["ClaimStartDt"])
df["ClaimEndDt"] = pd.to_datetime(df["ClaimEndDt"])

df["stay_len"] = (
    df["ClaimEndDt"] - df["ClaimStartDt"]
).dt.days.fillna(0)

In [30]:
df["Claim_After_Death"] = (
    df["DOD"].notna() &
    (df["ClaimStartDt"] > pd.to_datetime(df["DOD"]))
).astype(int)

In [31]:
df.sample()['stay_len']

415717    0
Name: stay_len, dtype: int64

In [32]:
df.drop(columns=["Note", "sentence_count"], inplace=True)

In [33]:
df.shape

(558211, 67)

In [ ]:
df.to_csv('before_aggregation.csv')

In [35]:
df.sample()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,is_inpatient,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,note_length,word_count,medical_term_count,general_word_count,short_note_flag,High_Cost_Flag,highcost_shortnote,diag_count,procedure_count,stay_len,Claim_After_Death
40419,BENE158965,CLM34233,2009-01-17,2009-01-24,PRV51244,4000,PHY416029,PHY371043,NaN,2009-01-17,78079,1068.0,2009-01-24,307,4280,4254,42731,496,53081,515,41400,2449,5990,NaN,9929.0,NaN,NaN,NaN,NaN,NaN,1.0,1934-08-01,NaN,1,1,0,4,730,12,12,1,1,2,2,1,1,1,1,1,1,2,16240,3204,1080,1270,75.0,1132,181,3,0,0,1,0,9,1,7,0


In [47]:
diag_cols = [col for col in df.columns if "ClmDiagnosisCode_" in col]
proc_cols = [col for col in df.columns if "ClmProcedureCode_" in col]

date_cols = [
    "ClaimStartDt",
    "ClaimEndDt",
    "AdmissionDt",
    "DischargeDt",
    "DOB",
    "DOD"
]

physician_cols = [
    "AttendingPhysician",
    "OperatingPhysician",
    "OtherPhysician"
]

text_cols = ["Note"]
# id_cols = ["ClaimID"]

columns_to_drop = (
    diag_cols +
    proc_cols +
    date_cols +
    physician_cols +
    text_cols
    # id_cols
)

claims = df.drop(columns=columns_to_drop, errors="ignore")

In [48]:
claims.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,is_inpatient,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,Age,note_length,word_count,medical_term_count,general_word_count,short_note_flag,High_Cost_Flag,highcost_shortnote,diag_count,procedure_count,stay_len,Claim_After_Death
0,BENE11001,CLM46614,PRV55912,26000,7866,1068.0,201,1.0,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,66.0,1309,207,1,2,0,1,0,9,0,6,0
1,BENE11001,CLM66048,PRV55907,5000,6186,1068.0,750,1.0,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,67.0,5078,880,5,1,0,1,0,3,1,2,0
2,BENE11001,CLM68358,PRV56046,5000,29590,1068.0,883,1.0,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,67.0,6977,1152,1,0,0,1,0,6,0,3,0
3,BENE11011,CLM38412,PRV52405,5000,431,1068.0,067,1.0,2,2,0,1,360,12,12,2,1,1,2,2,1,1,2,2,1,1,5000,1068,250,320,95.0,4473,700,5,2,0,1,0,9,1,8,0
4,BENE11014,CLM63689,PRV56614,10000,78321,1068.0,975,1.0,2,1,Y,45,780,12,12,2,1,1,2,1,1,2,1,2,2,2,21260,2136,120,100,71.0,2261,408,1,0,0,1,0,9,1,17,0


In [49]:
claims.shape

(558211, 42)

In [50]:
claims["Inpatient_Claim"] = claims["is_inpatient"]
claims["Outpatient_Claim"] = 1 - claims["is_inpatient"]

In [54]:
aggregation_dict = {
    "InscClaimAmtReimbursed": ["sum", "mean", "std", "max", "median", "min"],
    "DeductibleAmtPaid": ["sum", "mean"],
    "diag_count": "mean",
    "procedure_count": "mean",
    "stay_len": "mean",
    "Age": ["mean", "std"],

    "NoOfMonths_PartACov": "mean",
    "NoOfMonths_PartBCov": "mean",

    "ChronicCond_Alzheimer": "mean",
    "ChronicCond_Heartfailure": "mean",
    "ChronicCond_KidneyDisease": "mean",
    "ChronicCond_ObstrPulmonary":"mean",
    "ChronicCond_Depression": "mean",
    "ChronicCond_Diabetes": "mean",
    "ChronicCond_IschemicHeart": "mean",
    "ChronicCond_Osteoporasis": "mean",
    "ChronicCond_rheumatoidarthritis": "mean",
    "ChronicCond_stroke": "mean",

    "IPAnnualReimbursementAmt": "mean",
    "OPAnnualReimbursementAmt": "mean",
    "IPAnnualDeductibleAmt": "mean",
    "OPAnnualDeductibleAmt": "mean",
    
    "word_count": ["mean", "median"],
    "medical_term_count": "mean",
    "general_word_count": "mean",
    
    "short_note_flag": "mean",
    "highcost_shortnote": "mean",
    "High_Cost_Flag": "mean",
    "Claim_After_Death": "sum",

    "is_inpatient": "mean",
    
    "BeneID": "nunique",
    "ClaimID": "count",
    "Inpatient_Claim": "sum",
    "Outpatient_Claim": "sum"
}

In [55]:
df_provider = claims.groupby("Provider").agg(aggregation_dict)

In [56]:
df_provider.shape

(5410, 42)

In [57]:
df_labels = pd.read_csv('../data/Train-1542865627584.csv')

In [58]:
df_labels.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [59]:
df_provider.head()

InscClaimAmtReimbursed                                                \
                            sum         mean           std    max  median min   
Provider                                                                        
PRV51001                 104640  4185.600000  10796.091144  42000   400.0  10   
PRV51003                 605670  4588.409091   7309.794729  57000  1750.0   0   
PRV51004                  52170   350.134228    689.963754   3300    70.0   0   
PRV51005                 280910   241.124464    491.556392   4080    70.0   0   
PRV51007                  33710   468.194444   1433.769116  10000    80.0   0   

         DeductibleAmtPaid             diag_count procedure_count  stay_len  \
                       sum        mean       mean            mean      mean   
Provider                                                                      
PRV51001            5340.0  213.600000   3.200000        0.120000  1.440000   
PRV51003           66286.0  502.166667   5.250000        0.363636  3.674242   
PRV51004             310.0    2.080537   2.583893        0.000000  1.429530   
PRV51005            3700.0    3.175966   2.588841        0.000000  1.088412   
PRV51007            3264.0   45.333333   2.986111        0.013889  0.958333   

                Age            NoOfMonths_PartACov NoOfMonths_PartBCov  \
               mean        std                mean                mean   
Provider                                                                 
PRV51001  78.160000  11.915256           12.000000           12.000000   
PRV51003  69.590909  13.461823           11.818182           11.871212   
PRV51004  71.778523  12.918061           11.865772           11.959732   
PRV51005  70.082403  13.357434           11.907296           11.939914   
PRV51007  68.791667  14.628679           11.833333           11.833333   

         ChronicCond_Alzheimer ChronicCond_Heartfailure  \
                          mean                     mean   
Provider                                                  
PRV51001              1.400000                 1.240000   
PRV51003              1.575758                 1.393939   
PRV51004              1.570470                 1.409396   
PRV51005              1.634335                 1.416309   
PRV51007              1.638889                 1.444444   

         ChronicCond_KidneyDisease ChronicCond_ObstrPulmonary  \
                              mean                       mean   
Provider                                                        
PRV51001                  1.320000                   1.600000   
PRV51003                  1.515152                   1.689394   
PRV51004                  1.664430                   1.724832   
PRV51005                  1.564807                   1.746781   
PRV51007                  1.694444                   1.777778   

         ChronicCond_Depression ChronicCond_Diabetes  \
                           mean                 mean   
Provider                                               
PRV51001               1.640000             1.160000   
PRV51003               1.590909             1.242424   
PRV51004               1.577181             1.295302   
PRV51005               1.583691             1.314163   
PRV51007               1.597222             1.319444   

         ChronicCond_IschemicHeart ChronicCond_Osteoporasis  \
                              mean                     mean   
Provider                                                      
PRV51001                  1.080000                 1.760000   
PRV51003                  1.151515                 1.750000   
PRV51004                  1.275168                 1.671141   
PRV51005                  1.231760                 1.704721   
PRV51007                  1.291667                 1.708333   

         ChronicCond_rheumatoidarthritis ChronicCond_stroke  \
                                    mean               mean   
Provider                                                    

In [61]:
df_provider.columns = [
    "_".join(col).strip() if isinstance(col, tuple) else col
    for col in df_provider.columns
]
df_provider = df_provider.reset_index()

In [62]:
df_provider.to_csv('before_final.csv')